In [23]:
import openai

In [87]:
openai.api_key = 'sk-proj-'


In [25]:
with open('artykul.txt', 'r', encoding='utf-8') as file:
    content = file.read()
print(content)


Sztuczna inteligencja: wpĹyw i wyzwania

Sztuczna inteligencja to dziedzina nauki i technologii zajmujÄca siÄ tworzeniem maszyn i programĂłw zdolnych do wykonywania zadaĹ wymagajÄcych ludzkiej inteligencji, takich jak uczenie siÄ, rozumienie jÄzyka naturalnego i podejmowanie decyzji. AI staĹa siÄ integralnÄ czÄĹciÄ naszego codziennego Ĺźycia, od asystentĂłw gĹosowych w smartfonach, jak Siri czy Google Assistant, po systemy rekomendacyjne na platformach streamingowych, takich jak Netflix czy Spotify. Wspiera nas w planowaniu tras, automatyzacji domowych urzÄdzeĹ oraz w komunikacji. Obecnie jest o niej bardzo gĹoĹno chociaĹźby za sprawÄ duĹźych modeli jÄzykowych, jak ChatGPT. 

RozwĂłj uczenia maszynowego i gĹÄbokiego uczenia umoĹźliwiĹ tworzenie zaawansowanych modeli, ktĂłre potrafiÄ samodzielnie rozwiÄzywaÄ skomplikowane problemy. Sieci neuronowe analizujÄ ogromne iloĹci danych w obszarach takich jak rozpoznawanie obrazĂłw czy przetwarzanie jÄzyka naturalne

In [4]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",  
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What are the benefits of renewable energy?"},
    ],
    max_tokens=100,
    temperature=0.7,
)

print(response["choices"][0]["message"]["content"])


Renewable energy sources have numerous benefits, including:

1. Environmental sustainability: Renewable energy sources like solar, wind, and hydropower produce minimal greenhouse gas emissions, helping to combat climate change and reduce air and water pollution.

2. Energy security: By diversifying our energy sources and reducing our reliance on fossil fuels, renewable energy can enhance energy security and reduce the risks associated with volatile fuel prices and supply disruptions.

3. Economic growth and job creation: The renewable energy sector has the potential to


In [27]:

import logging

# Konfiguracja logowania błędów
logging.basicConfig(
    filename="error_log.txt",
    level=logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def load_api_key():
    """
    Load API key from an environment variable for better security.
    """
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise EnvironmentError("OpenAI API key not found in environment variables.")
    return api_key

# Inicjalizacja klucza API
try:
    openai.api_key = load_api_key()
except EnvironmentError as e:
    logging.error(e)
    raise


In [28]:
def validate_and_load_file(file_path):
    """
    Validate the file and load its content.
    """
    if not os.path.exists(file_path):
        error_msg = f"File '{file_path}' does not exist."
        logging.error(error_msg)
        raise FileNotFoundError(error_msg)
    if os.path.getsize(file_path) == 0:
        error_msg = f"File '{file_path}' is empty."
        logging.error(error_msg)
        raise ValueError(error_msg)
    
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read().strip()
            if not content:
                raise ValueError(f"File '{file_path}' contains no valid data.")
            return content
    except Exception as e:
        logging.error(f"Failed to read file '{file_path}': {e}")
        raise


In [36]:
def split_content(content, max_length=2000):
    """
    Split content into smaller chunks to fit within token limits.
    """
    paragraphs = content.split("\n\n")
    chunks = []
    current_chunk = []
    current_length = 0

    for paragraph in paragraphs:
        paragraph_length = len(paragraph)
        if current_length + paragraph_length > max_length:
            chunks.append("\n\n".join(current_chunk))
            current_chunk = [paragraph]
            current_length = paragraph_length
        else:
            current_chunk.append(paragraph)
            current_length += paragraph_length
    if current_chunk:
        chunks.append("\n\n".join(current_chunk))
    return chunks


In [49]:
def call_openai_api(prompt, max_tokens=1500):
    """
    Call the OpenAI API and handle potential errors with validation.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an assistant that generates HTML code based on user input."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=0.7,
        )
        if "choices" not in response or not response["choices"]:
            raise ValueError("Invalid response from OpenAI API.")
        return response["choices"][0]["message"]["content"]
    except openai.error.OpenAIError as e:
        logging.error(f"OpenAI API call failed: {e}")
        raise
    except Exception as e:
        logging.error(f"Unexpected error during API call: {e}")
        raise


In [50]:
def save_to_file(content, file_path):
    """
    Save the content to a file.
    """
    try:
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)
        print(f"Content saved to '{file_path}'.")
    except Exception as e:
        logging.error(f"Failed to save file '{file_path}': {e}")
        raise


In [89]:
def main():
    try:
        # Ścieżka do pliku wejściowego
        article_file_path = "artykul.txt"

        # Walidacja i wczytanie pliku
        article_content = validate_and_load_file(article_file_path)

        # Podział treści na fragmenty
        chunks = split_content(article_content)

        # Przetwarzanie każdego fragmentu
        for i, chunk in enumerate(chunks):
            prompt = f"""
Przygotuj kod HTML dla poniższego artykułu, uwzględniając następujące wytyczne:

1. **Struktura dokumentu HTML**:
   - Kod HTML powinien zawierać semantyczne znaczniki, takie jak <article>, <section>, <header>, <footer>, <h1>, <h2>, itp.
   - Podziel artykuł na sekcje z nagłówkami (<h2>) odpowiadającymi głównym podtytułom w treści.
   - Wyróżnij pierwsze zdanie artykułu jako wprowadzenie, używając znacznika <strong> lub <em>, jeśli odpowiednie.

2. **Grafiki i multimedia**:
   - Dodaj miejsca na grafiki, używając tagów <img>. Ustaw wartość atrybutu src="image_placeholder.jpg".
   - Dodaj do obrazków atrybut alt opisujący, co powinno znajdować się na obrazie (np. "Wykres przedstawiający dane sprzedażowe za 2023 rok").
   - Pod każdą grafiką dodaj podpis w znaczniku <figcaption>, np. "Rysunek przedstawiający kluczowe dane statystyczne".

3. **Listy i wypunktowania**:
   - Jeśli artykuł zawiera listy (np. punkty lub kroki), użyj odpowiednich tagów HTML, takich jak <ul> lub <ol>, oraz <li> dla elementów listy.

4. **Linki i odniesienia**:
   - Jeśli w tekście są odniesienia do innych źródeł, zamień je na działające linki HTML, używając znacznika <a href="URL">.

5. **Stylizacja**:
   - Zastosuj znaczniki semantyczne, takie jak <blockquote>, <code>, czy <pre>, jeśli tekst zawiera cytaty, fragmenty kodu lub dane w formacie tabelarycznym.
   - W przypadku cytatów dodaj atrybut cite w znaczniku <blockquote> z przykładowym źródłem.

6. **Tabela zawartości**:
   - Dodaj na początku artykułu tabelę zawartości (<nav>) z linkami do sekcji, wykorzystując znaczniki <a href="#id_sekcji">.

7. **Atrybuty dostępności (Accessibility)**:
   - Upewnij się, że wszystkie tagi <img> mają odpowiednie opisy w atrybucie alt.
   - Jeśli występują tabele, dodaj znaczniki <thead>, <tbody> i odpowiednie nagłówki <th>.

8. **Treść artykułu**:
   - Kod HTML powinien zawierać tylko zawartość między <body> i </body>, bez <html>, <head> lub <body>.
   - Cały tekst artykułu poniżej.

Oto artykuł:
{chunk}
"""

            html_output = call_openai_api(prompt)

            # Zapisanie wyników
            output_file = f"artykul_part_{i+1}.html"
            save_to_file(html_output, output_file)

    except Exception as e:
        logging.error(f"Critical error in main process: {e}")
        print(f"An error occurred: {e}")


In [85]:
def test_validate_and_load_file():
    # Test dla istniejącego pliku
    try:
        content = validate_and_load_file("test_file.txt")  # Plik testowy
        assert content, "File content should not be empty"
        print("File validation test passed.")
    except Exception as e:
        print(f"File validation test failed: {e}")

def test_split_content():
    # Test dla funkcji split_content
    content = "Paragraph 1.\n\nParagraph 2.\n\nParagraph 3."
    chunks = split_content(content, max_length=15)
    assert len(chunks) == 3, "Content should split into 3 chunks."
    print("Content splitting test passed.")


In [88]:
if __name__ == "__main__":
    main()


An error occurred: Incorrect API key provided: sk-proj-. You can find your API key at https://platform.openai.com/account/api-keys.
